In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)
import lightautoml

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import multiprocessing as mp

# set option of max 100 displayed rows
pd.set_option('display.max_rows', 200)

In [ ]:
!pip install lightautoml

     |████████████████████████████████| 293 kB 5.1 MB/s 
     |████████████████████████████████| 308 kB 38.0 MB/s 
     |████████████████████████████████| 1.2 MB 11.8 MB/s 
     |████████████████████████████████| 76.3 MB 19 kB/s 
     |████████████████████████████████| 424 kB 54.8 MB/s 
     |████████████████████████████████| 804.1 MB 2.7 kB/s 
     |████████████████████████████████| 204 kB 76.8 MB/s 
     |████████████████████████████████| 15.7 MB 59 kB/s 
     |████████████████████████████████| 2.8 MB 14.6 MB/s 
     |████████████████████████████████| 209 kB 83.6 MB/s 
     |████████████████████████████████| 80 kB 10.3 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 112 kB 76.3 MB/s 
     |████████████████████████████████| 149 kB 80.3 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 23.3 MB 1.3 MB/s 
     |████████████████████████████████| 22.1 MB 1.3 MB/s 
     |███████████████

In [ ]:
train_df = pd.read_csv('/content/train_dataset_hackathon_mkb.csv',sep=';',encoding='cp1251')

In [ ]:
test_df = pd.read_csv('/content/test_dataset_hackathon_mkb.csv',sep=';',encoding='cp1251')

In [ ]:
N_THREADS = mp.cpu_count() # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML - 10 is better
RANDOM_STATE = 42 # fixed random state for various reasons
TIMEOUT = 3600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

In [ ]:
%%time

roles = {'target': TARGET_NAME,
         'drop': ['id_contract','id_client']
         }

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 13.4 µs


In [ ]:
%%time

task = Task('binary', loss='logloss', metric='auc')

CPU times: user 7.7 ms, sys: 0 ns, total: 7.7 ms
Wall time: 19.5 ms


In [ ]:
#### %%time 
automl = TabularAutoML(task = task, 
                       timeout = 2400,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(train_df, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Выходные данные были обрезаны до нескольких последних строк (5000).
[2021-11-21 22:12:03,604] (DEBUG): 56:	test: 0.9436407	best: 0.9436407 (56)	total: 1.01s	remaining: 52.1s
[2021-11-21 22:12:03,620] (DEBUG): 57:	test: 0.9438018	best: 0.9438018 (57)	total: 1.02s	remaining: 51.9s
[2021-11-21 22:12:03,638] (DEBUG): 58:	test: 0.9438301	best: 0.9438301 (58)	total: 1.04s	remaining: 51.8s
[2021-11-21 22:12:03,668] (DEBUG): 59:	test: 0.9439577	best: 0.9439577 (59)	total: 1.07s	remaining: 52.3s
[2021-11-21 22:12:03,689] (DEBUG): 60:	test: 0.9441375	best: 0.9441375 (60)	total: 1.08s	remaining: 52.1s
[2021-11-21 22:12:03,706] (DEBUG): 61:	test: 0.9442728	best: 0.9442728 (61)	total: 1.09s	remaining: 51.9s
[2021-11-21 22:12:03,723] (DEBUG): 62:	test: 0.9441556	best: 0.9442728 (61)	total: 1.11s	remaining: 51.8s
[2021-11-21 22:12:03,741] (DEBUG): 63:	test: 0.9442702	best: 0.9442728 (61)	total: 1.13s	remaining: 51.7s
[2021-11-21 22:12:03,754] (DEBUG): 64:	test: 0.9445332	best: 0.9445332 (64)	total: 1

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(train_df['TARGET'],oof_pred.data)

0.9658399978927639

In [ ]:
df_otl = pd.DataFrame()

In [ ]:
top3 = pd.read_csv('top3.csv')

In [ ]:
konh = pd.read_csv('/content/konh.csv')

In [ ]:
form = konh[['id_client','TARGET']]

In [ ]:
form_dict = dict({})
for x in top3['TOP>2']:
    form_dict[x] = form[form['id_client']==x]['TARGET'].array[0]

In [ ]:
top3 = list(top3['TOP>2'].array)

In [ ]:
df_otl['id_client'] = test_df['id_client']
df_otl['id_contract'] = test_df['id_contract']


In [ ]:
idc = list(df_otl['id_client'].array)

In [ ]:
test_pred = automl.predict(test_df)

In [ ]:
test_pred = list(test_pred.data.squeeze())

In [ ]:
tar = test_pred
for i in range(7330):
    if idc[i] in top3:
        tar[i] = form_dict[idc[i]]

In [ ]:
df_otl['TARGET'] = tar

In [ ]:
df_otl = df_otl.drop(['id_client'],axis=1)

In [ ]:
automl.get_feature_scores()

,Feature,Importance
0,DATEFIRSTREG,80675.291219
1,F1100,30135.410768
2,OKVED_CODE,22026.928375
3,WORKERSRANGE,14368.659729
4,TELEPHONECOUNT,12835.159078
5,TAXREGPAY_REGDATE,7753.734282
6,F2120,6532.592033
7,OKTMO_CODE,5430.493891
8,SIGN_DATE,5177.524917
9,SIGNEDNUMBER_95_EVER,4989.866630


In [ ]:
df_otl.to_csv('trying_22_last.csv',sep=';',index=False)